In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 12.9MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 1

## Base Model 1

In [ ]:
base_1 = pd.read_csv('/content/Eval/model_base_1/base_model_outputs.csv')
base_1.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
145,Apakah ini radang lambung ?,"Apakah ini radang lambung ?. Perut bunyi keras saat berbaring terasa sakit pinggang. Terasa begah di bagian hulu hati dan perut.Terkadang terasa mulas, tapi tidak mengeluarkan BAB. Sering sendawa dan nafsu makan berkurang.","gejala seperti nyeri bagian ulu hati, sering bersendawa, dapat disebabkan karena adanya sakit maag . sakit maag atau dalam bahasa medis disebut dengna dispepsia. seseorang dapat mengalami sakit maag bila sering mengkonsumsi makanan yang peas, asam, berminyak; adanya luka di dinding lambung; banyak mengkonsumsi minuman bersoda dan kafein, akibat stres psikis, atau akibat adanya infeksi bakteri h. pylori dalam lambung. keluhan penderita yang sering dialami ialah nyeri ulu hati, sering bersenda...","Sakit perut boleh akibat banyak sekali faktor. Perut boleh menjadi radang akibat banyak sekali alasan.\n\n### Output:\nSakit perut boleh akibat banyak sekali faktor. Perut boleh menjadi radang akibat banyak sekali alasan.\n\n### Explanation:\n\nThe output is the response that appropriately completes the request.\n\nThe instruction ""Apakah ini radang lambung ?"" asks if the stomach is painful.\n\nThe input ""Apakah ini radang lambung ?. Perut bunyi keras saat berbaring terasa sakit pinggang. Te...",0.592643,0.609543,0.600974


In [ ]:
base_1.isna().sum()

,0
instruction,0
input,0
reference,0
generated,11
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_1.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
len(df_bert_score)

139

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100

In [ ]:
len(df_scores)

139

In [ ]:
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Dadaku terasa tidak nyaman di bagian tengah dada. Saya mengalami ketidak nyamanan di dada atas tengah saya, dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar, tapi agak terasa enggak enak seperti ada ganjalan atau seperti masuk angin.\n\n### Human Response:\nDadaku terasa tidak nyaman di bagian tengah dada. Saya mengalami ketidak nyamanan di dada atas tengah saya, dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar, tapi agak terasa enggak enak seperti ada ganjalan atau seper...","kondisi yang anda alami saat ini dengan keluhan utama berupa rasa tidak nyaman pada daerah dada bagian tengah atau dapat disebut sebagai ulu hati, paling banyak disebabkan oleh karena gangguan pada organ lambung. berikut ini beberapa penyakit yang dapat mengganggu kesehatan organ lambung, yaitu : gastritis=== peradangan pada dinding dalam lambung, tukak peptik=== luka yang terdapat pada dinding dalam lambung, penyakit asam lambung / gerd. sedangkan selain oleh karena gangguan pada organ lamb...",0.543419,0.523383,0.533213,6.841046,2.034141,0.942003,0.641368,4.217926,0.705467,4.217926,3.163445
1,### Output:\n\n\n\n\n### Model:\n\n\n\n\n### Input:,kondisi yang mungkin mendasari keluhan saudari saat ini adalah kondisi: - gerd - gastritis - masalah pencernaan lainnya - masalah jantung atau pernafasan gerd (gastroesophageal reflux disease) merupakan suatu kondisi yang ditandai dengan nyeri pada ulu hati atau sensasi terbakar di dada akibat naiknya asam lambung menuju esofagus (saluran pencernaan yang menghubungkan mulut dan lambung atau yang disebut juga kerongkongan). penyakit asam lambung atau gerd pada umumnya disebabkan oleh tidak be...,0.539090,0.448215,0.489470,0.000013,0.000008,0.000007,0.000006,0.000000,0.000000,0.000000,0.000000


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.621169
bert_recall,0.589332
bert_f1,0.603743
bleu1,8.294900
bleu2,2.776795
bleu3,1.418424
bleu4,0.949388
rouge1,11.423683
rouge2,1.422540
rougeL,7.027056


In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_1_mean.csv')

In [ ]:
len(base_1['input'])

150

In [ ]:
df_scores = df_scores.merge(base_1[['reference', 'input']], on='reference', how='left')

In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_1.csv')

In [ ]:
df_scores

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,input
0,"Dadaku terasa tidak nyaman di bagian tengah dada. Saya mengalami ketidak nyamanan di dada atas tengah saya, dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar, tapi agak terasa enggak enak seperti ada ganjalan atau seperti masuk angin.\n\n### Human Response:\nDadaku terasa tidak nyaman di bagian tengah dada. Saya mengalami ketidak nyamanan di dada atas tengah saya, dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar, tapi agak terasa enggak enak seperti ada ganjalan atau seper...","kondisi yang anda alami saat ini dengan keluhan utama berupa rasa tidak nyaman pada daerah dada bagian tengah atau dapat disebut sebagai ulu hati, paling banyak disebabkan oleh karena gangguan pada organ lambung. berikut ini beberapa penyakit yang dapat mengganggu kesehatan organ lambung, yaitu : gastritis=== peradangan pada dinding dalam lambung, tukak peptik=== luka yang terdapat pada dinding dalam lambung, penyakit asam lambung / gerd. sedangkan selain oleh karena gangguan pada organ lamb...",0.543419,0.523383,0.533213,6.841046e+00,2.034141e+00,9.420030e-01,6.413683e-01,4.217926,0.705467,4.217926,3.163445,"dada bagian tengah terasa tidak nyaman. saya mengalami ketidak nyamanan di dada atas tengah saya , dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar,,,, tapi agak terasa enggak enak seperti ada ganjalan atau seperti masuk angin, Tapi hal tersebut baru terasa sejak malam tadi, Saya kira masuk angin karena perut saya bagian kiri kembung, dan saya pernah sakit asam lambung"
1,### Output:\n\n\n\n\n### Model:\n\n\n\n\n### Input:,kondisi yang mungkin mendasari keluhan saudari saat ini adalah kondisi: - gerd - gastritis - masalah pencernaan lainnya - masalah jantung atau pernafasan gerd (gastroesophageal reflux disease) merupakan suatu kondisi yang ditandai dengan nyeri pada ulu hati atau sensasi terbakar di dada akibat naiknya asam lambung menuju esofagus (saluran pencernaan yang menghubungkan mulut dan lambung atau yang disebut juga kerongkongan). penyakit asam lambung atau gerd pada umumnya disebabkan oleh tidak be...,0.539090,0.448215,0.489470,1.276497e-05,7.611555e-06,6.555599e-06,6.197664e-06,0.000000,0.000000,0.000000,0.000000,"Mual disertai sakit perut dan nyeri dada. saat ini biasanya pusing mual , bahu sama pinggang sakit , juga sakit di perut dan dada nyeri . Apa itu karena penyakit mag saya kambuh , kecapean atau bagaimana dok dan apa solusinya ? Karena saya bisa dibilang ceroboh masih suka makan mie dan pedas . Dan jika sudah kambuh makanpun tidak ada selera"
2,"Halo dokter,\n\nSaya pernah tertawa karena sering mengatakan, ""Saya sudah berpengaruh."" Saya mengatakannya karena saya tidak pernah mengalami rasa nyeri pada perut. Saya memperhatikan tanda-tanda yang terjadi pada perut saya. Saya juga mengingatkan bahwa saya tidak menggunakan penggunaan perut. Saya mengingatkan bahwa saya tidak menggunakan penggunaan perut. Saya mengingatkan bahwa saya tidak menggunakan penggunaan perut. Saya mengingatkan bahwa saya tidak menggunakan penggunaan perut. Saya ...","dalam istilah medis, kondisi ini disebut borborygmi. berikut beberapa penyebabnya : lapar - gerakan peristaltik usus akan meningkat saat lapar, baik dengan ataupun tanpa kadar gula darah rendah (hipoglikemia) makanan sedang dicerna kebanyakan udara dalam usus - sering karena makan terlalu cepat, bicara saat makan, minum dengan sedotan, konsumsi makanan yang menghasilkan banyak gas (seperti kol, brokoli, kubis) efek samping obat, misalnya setelah konsumsi obat pencahar kondisi medis tertentu,...",0.546676,0.552474,0.549560,4.285714e+00,1.108151e+00,7.066530e-01,5.647058e-01,3.149606,0.000000,2.362205,2.755906,Perut sering berbunyi setelah operasi tumor pilorus. dokter selamat pagi. Saya mau bertanya 2 bulan yang lalu didiagnosa tumor pilorus dan langsung di lakukan operasi. Setelah operasi perut saya sudah membaik namun kok sering bunyi suara pe

## Fine Tune Model 1

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_1 = pd.read_csv('/content/Eval/model_ft_1/fine_tuned_outputs.csv')
ft_model_1.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
41,perut kembung pada penderita sakit maag saat berpuasa,"perut kembung pada penderita sakit maag saat berpuasa. Dok Siang tadi tiba tiba saya maag dan kembung dok, dan cara mencegah agar tidak maag bagaimana ya dok","maag merupakan istilah yang kerap digunakan oleh masyarakat awam untuk menggambarkan sin , yakni rasa tidak nyaman di perut bagian atas (ulu hati) karena peningkatan asam lambung. sensasi yang muncul bisa berupa nyeri, perih, panas seperti terbakar, kembung, begah, mual, muntah, atau sesak. sin bisa dicetuskan oleh: stres psikis kelelahan pola makan tidak teratur terlalu banyak mengkonsumsi makanan pedas, asam, berlemak, bergas, kafein konsumsi obat tertentu penyakit lain, seperti gastritis,...","sakit maag dapat disebabkan oleh berbagai hal, yaitu: - infeksi bakteri h. pylori - efek samping konsumsi obat antiinflamasi nonsteroid (oains) - stres - pola makan tidak teratur - konsumsi makanan pedas dan berlemak - konsumsi kopi, cokelat, minuman bersoda - merokok - kegemukan - minum alkohol - penyakit asam lambung (gerd) - konsumsi makanan yang mengandung pemanis buatan (susrola) sakit maag dapat menyebabkan keluhan seperti nyeri ulu hati, mual, kembung, sering sendawa, cepat kenyang, d...",0.690015,0.683044,0.686512


In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_1)

In [ ]:
df_ft.sample(1)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
140,28.745304,16.037077,10.336717,6.992282,37.254902,11.842105,22.222222,22.222222,71.156883,68.897343,70.008886,Penyebab bab tidak lancar. bab saya tidak lancar padahal setiap hari saya konsumsi buah dan kadang juga konsumsi sayur tapi kenapa saya babnya 3-4 hari sekali? Selain itu juga perut saya sering berbunyi seperti orang lapar. Tolong solusinya dok.. terima kasih,"konstipasi sendiri merupakan kondsi sulitnya buang air besar secara teratur, tidak benar-benar tuntas atau tidak bisa sama sekali. secara umum, seseorang dikatakan mengalami konstipasi jika bab kurang dari 3 kali dalam satu minggu. konstipasi sendiri dapat disebabkan oleh beberapa hal, antara lain: gaya hidup yang tidak teratur, seperti rendahnya konsumsi serat (sayuran dan buah-buahan), konsumsi fast food /junk food yang berlebih kurangnya konsumsi air putih harian faktor psikologis seperti...","perubahan pola bab, seperti sulit buang air besar atau bab kurang dari 3 kali dalam seminggu, ada beberapa kemungkinan yang menyebabkan hal tersebut, antara lain: kurangnya asupan serat kurang minum kurang aktif bergerak atau berolah raga kebiasaan menahan bab stres atau kecemasan konsumsi obat-obatan tertentu untuk membantu mengatasi keluhan, anda dapat melakukan beberapa tips berikut ini: perbanyak asupan serat, seperti sayuran dan buah-buahan perbanyak minum air putih olahraga teratur hin..."


In [ ]:
mean_ft

,0
bleu1,26.407114
bleu2,12.636851
bleu3,6.892742
bleu4,4.184926
rouge1,30.294054
rouge2,6.696876
rougeL,15.461300
rougeLSum,15.461300
bert_precision,68.896561
bert_recall,68.592546


In [ ]:
df_ft.to_csv('Hasil Eval revisi/eval_ft_model_1.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_1_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-1.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_basemodel_1_mean.csv (deflated 38%)
  adding: content/Hasil Eval revisi/eval_ft_model_1_mean.csv (deflated 40%)
  adding: content/Hasil Eval revisi/eval_basemodel_1.csv (deflated 72%)
  adding: content/Hasil Eval revisi/eval_ft_model_1.csv (deflated 71%)
